WeatherPY

In [8]:
# dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import requests
from scipy.stats import linregress
from citipy import citipy
import json
import time
import os
from datetime import datetime






In [ ]:
# import the OpenWeatherMap API key
import open_weather_API_key

# define the base URL for the OpenWeatherMap API
base_url = "http://api.openweathermap.org/data/2.5/weather?"

# define the units for the temperature
units = "metric"

# define the units for the humidity
humidity_units = "%"

# define the units for the wind speed
wind_units = "m/s"

# define the units for the cloudiness
cloud_units = "%"



Generate the Cities List by Using citipy Library

In [ ]:
# empty list for holding the latitude and longitude combinations
lat_lngs = []

# empty list for holding the cities
cities = []

# range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# create a set of random latitude and longitude combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# identify the nearest city for each latitude and longitude combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # if the city is unique, then add it to the cities list
    if city not in cities:
        cities.append(city)

# print the city count to confirm sufficient count
print(len(cities))

Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# set the API base URL
base_url = "http://api.openweathermap.org/data/2.5/weather?"

# set the units for the temperature
units = "metric"

# set the units for the humidity
humidity_units = "%"

# set the units for the wind speed
wind_units = "m/s"

# set the units for the cloudiness
cloud_units = "%"

# create an empty list to hold the weather data for each city
city_data = []

# print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# create counters
record_count = 1
set_count = 1

# loop through all the cities in the list to fetch weather data
for i, city in enumerate(cities):
        
    # group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # create the endpoint URL
    city_url = f"{base_url}q={city}&appid={open_weather_API_key.api_key}&units={units}"

    # log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # add 1 to the record count
    record_count += 1

    # run an API request for each of the cities
    try:
        # parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # parse out latitude, longitude, maximum temperature, humidity, cloudiness, wind speed, country and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_country = city_weather["sys"]["country"]
        city_wind_speed = city_weather["wind"]["speed"]
        city_clouds = city_weather["clouds"]["all"]
        city_date = city_weather["dt"]

    # append the city information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind_speed,
                          "Country": city_country,
                          "Date": city_date})
    
    # if an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# indicate that data loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# convert the city data into a DataFrame
city_data_df = pd.DataFrame(city_data)

# show the record count
city_data_df.count()

In [ ]:
# display the sample data
city_data_df.head(10)

In [ ]:
# export the city data into a CSV
city_data_df.to_csv("WeatherPy_Database.csv", index_label="City_ID")

In [ ]:
# read saved data
city_data_df = pd.read_csv("WeatherPy_Database.csv", index_col="City_ID")

# display sample data
city_data_df.head(10)